In [6]:
from omegaconf import OmegaConf

In [7]:
from utils.config import get_class_from_str

device = 'cuda'
config = OmegaConf.load('../config/mnist.yml')
model = get_class_from_str(config.model.target)(**config.model.params).to(device)
encoder = get_class_from_str(config.encoder.target)(**config.encoder.params).to(device)
diffusion = get_class_from_str(config.diffusion.target)(
        model, **config.diffusion.params, latent_encoder=encoder
).to(device)

UNet
resolution : 28 for level 0. Attentions: False
resolution: 14 for level 2. Attentions: True
resolution: 28 for level 0. Attentions: False
LatentEncoder
level 0. Attentions: False
level 0. Attentions: True


In [8]:
import torch

diffusion.load_state_dict(torch.load('../runs/mnist_big_lr/diffusion_23433.pt')['model_state_dict'])

<All keys matched successfully>

In [14]:
from scripts.train import DatasetWrapper

diffusion.eval()
dataset = DatasetWrapper(get_class_from_str(config.data.target)(**config.data.params))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)

In [15]:
mini_batch = next(iter(dataloader))
x_1 = mini_batch[0][0].to(device)
x_2 = mini_batch[0][1].to(device)

In [12]:
steps = 100

In [16]:
import numpy as np
from PIL.Image import LANCZOS
from torchvision.transforms import ToPILImage
import imageio

x_1_latent = diffusion.latent_encoder(x_1[None])
x_2_latent = diffusion.latent_encoder(x_2[None])


interpolations = torch.stack([torch.lerp(x_1_latent, x_2_latent, t) for t in torch.linspace(0, 1, steps=steps).to(x_1.device)]).squeeze(1)

noise = torch.randn((steps, model.in_channels, *model.size)).to(x_1.device)
y_s = diffusion.p_decode_loop((steps, model.in_channels, *model.size), interpolations.squeeze(1), x_start=noise)
y_s = torch.clamp(y_s, 0, 1)
video = imageio.get_writer('interpolation.gif', fps=10)
for y in y_s:
    y = ToPILImage()(y.cpu())
    y = y.resize((64, 64), LANCZOS)
    video.append_data(np.array(y))
for y in reversed(y_s):
    y = ToPILImage()(y.cpu())
    y = y.resize((64, 64), LANCZOS)
    video.append_data(np.array(y))
video.close()

sampling loop time step: 100%|██████████| 1000/1000 [00:14<00:00, 68.81it/s]
